In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [2]:
%%capture
!pip install pandas tiktoken graphrag

In [1]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [3]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [4]:
# parquet files generated from indexing pipeline
INPUT_DIR = "/workspaces/graphrag/nm-projects/asai/output/20240716-153628/artifacts/"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [5]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

/usr/local/python/3.10.13/lib/python3.10/site-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/usr/local/python/3.10.13/lib/python3.10/site-packages/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Report records: 137


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,141,# GPT and Related Entities\n\nThe community re...,4,8.5,GPT and Related Entities,The impact severity rating is high due to the ...,"The community revolves around GPT, a versatile...",[{'explanation': 'Nikhil Maddirala provides de...,"{\n ""title"": ""GPT and Related Entities"",\n ...",a7fd871a-7311-4384-86ac-730b43f9ab3f
1,140,# Nikhil Maddirala and AI Technology\n\nThe co...,4,8.5,Nikhil Maddirala and AI Technology,The impact severity rating is high due to Nikh...,The community revolves around Nikhil Maddirala...,[{'explanation': 'Nikhil Maddirala is a key fi...,"{\n ""title"": ""Nikhil Maddirala and AI Techn...",3379911d-8a50-4339-a5a4-58e1b1259711
2,129,# Linear Regression and Home Price Prediction\...,3,7.5,Linear Regression and Home Price Prediction,The impact severity rating is high due to the ...,The community revolves around the topics of Li...,[{'explanation': 'Nikhil Maddirala demonstrate...,"{\n ""title"": ""Linear Regression and Home Pr...",9fd58f28-de2a-463a-ac02-a9fdcfdac77c
3,131,# Neural Networks and Discussions\n\nThe commu...,3,7.5,Neural Networks and Discussions,The impact severity rating is high due to the ...,The community revolves around discussions rela...,[{'explanation': 'Nikhil Maddirala is a key fi...,"{\n ""title"": ""Neural Networks and Discussio...",abcf7478-b4a6-42cb-a382-2f4e47a1749f
4,133,# Machine Learning and Nikhil Maddirala\n\nThe...,3,7.5,Machine Learning and Nikhil Maddirala,The impact severity rating is high due to the ...,The community revolves around the concept of M...,[{'explanation': 'Nikhil Maddirala delves into...,"{\n ""title"": ""Machine Learning and Nikhil M...",df56cb58-347c-43e9-ae6a-dc1d9f85be8c


#### Build global context based on community reports

In [6]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [8]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [14]:
result = await search_engine.asearch(
    "Give me a full outline of this entire podcast including the main topics and key takeaways."
)

response = result.response

from IPython.display import display, HTML
display(HTML(f"<p style='width: 70ch;'>{response}</p>"))

In [9]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

NameError: name 'result' is not defined

In [20]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 6. LLM tokens: 56271


# Graph viz


## yfiles_jupyter_graphs

Source: https://github.com/microsoft/graphrag/issues/418

In [1]:
%%capture
!pip install yfiles_jupyter_graphs==1.7.3 pandas notebook ipywidgets

In [1]:
import pandas as pd
entity_df = pd.read_parquet("/workspaces/graphrag/nm-projects/asai/output/20240716-171659/artifacts/create_final_nodes.parquet")
relationship_df = pd.read_parquet("/workspaces/graphrag/nm-projects/asai/output/20240716-171659/artifacts/create_final_relationships.parquet")


In [2]:
"""
Uses yfiles-jupyter-graphs to visualize the dataframes.

The dataframes are converted into supported nodes and relationships lists and then passed to yfiles-jupyter-graphs.
Additionally, some values are mapped to visualization properties.
"""

import pandas as pd

def show_graph(entity_df, relationship_df):
    from yfiles_jupyter_graphs import GraphWidget

    # converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_entities_to_dicts(df):
        nodes_dict = {}
        for index, row in df.iterrows():
            # Create a dictionary for each row and collect unique nodes
            node_id = row["title"]
            if not node_id in nodes_dict:
                nodes_dict[node_id] = {"id": node_id, "properties": {col: value for col, value in zip(row.index, row.values)}}
        return list(nodes_dict.values())

    # converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_relationships_to_dicts(df):
        relationships = []
        for index, row in df.iterrows():
            # Create a dictionary for each row
            relationships.append({"start": row['source'], "end": row['target'], "properties": {col: value for col, value in zip(row.index, row.values)}})
        return relationships


    w = GraphWidget()
    # use the converted data to visualize the graph
    w.nodes = convert_entities_to_dicts(entity_df)
    w.edges = convert_relationships_to_dicts(relationship_df)
    w.directed = True
    # show title on the node
    w.node_label_mapping = "title"
    # map community to a color
    def community_to_color(community):
        colors = ["crimson", "darkorange", "indigo", "cornflowerblue", "cyan", "teal", "green"]
        return colors[int(community) % len(colors)] if community is not None else "lightgray"
    def edge_to_source_community(edge):
        source_node = next((entry for entry in w.nodes if entry["properties"]["title"] == edge["start"]), None)
        source_node_community = source_node["properties"]["community"]
        return source_node_community if source_node_community is not None else None
    w.node_color_mapping = lambda node : community_to_color(node["properties"]["community"])
    w.edge_color_mapping = lambda edge : community_to_color(edge_to_source_community(edge))
    # map size data to a reasonable factor
    w.node_scale_factor_mapping = lambda node : 0.5 + node["properties"]["size"] * 1.5 / 20
    # use weight for edge thickness
    w.edge_thickness_factor_mapping = "weight"
    # Use the circular layout for this visualization. For larger graphs, the default organic layout is often preferable.
    w.circular_layout()
    display(w)

show_graph(entity_df, relationship_df)

GraphWidget(layout=Layout(height='800px', width='100%'))